In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
from qx_noface_no2d_video_swin_transformer01 import SwinTransformer3D
import gc
from einops import rearrange
from tqdm import tqdm
# from kw_r2plus1d_video_swin_transformer import SwinTransformer3D
batchsz = 16
start_epoch = 0
test_flage=False
model_name ="15Frame VAA Fusion in stage 3 dim=128 head4,8,16,32"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()
# device = torch.device('cuda:1' if USE_CUDA else 'cpu')

In [2]:
logger.debug('============={}+Video swin transformerTrain Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2024-01-16 10:45:21,168]::<ipython-input-2-8c4f50b4ea24>::DEBUG::=============15Frame VAA Fusion in stage 3 dim=128 head4,8,16,32+Video swin transformerTrain Strat!=============


True
학습을 진행하는 기기: cuda:0


In [3]:
def select_frame_data(datas,frame_num):
    data_set=[]
    begin_num=0
    x=128/frame_num
    indexes=[None] * frame_num
    for i in range(frame_num):
        indexes[i]=int(begin_num)
        begin_num+=x
    for data in datas:
        video_frames = [None] * 2
        video_frames[0]=[]
        video_frames[1]=[]
        for index in indexes:
            video_frames[0].append(data[0][index])
        video_frames[1].append(data[1])
        data_set.append(video_frames)
        del video_frames
        del data 
        gc.collect()
    del datas
    gc.collect()
    return data_set
print(1)

1


In [4]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(i)
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

1


In [5]:
def datasize(type,batchsz):
    if type=='train':
        i=5978#5967
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
    else:
        i=1992#1986
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
print(1)

1


In [6]:

save_model_file_path = './save_swintransformer_folder/{}_{}.{}'
mtcnn= MTCNN(image_size=224, margin=0, min_face_size= 20, thresholds=[0.6, 0.7, 0.7], post_process=True)
# test_set_data=getdata("/home/ssrlab/qx/code/test/video-swin-transformer-pytorch/data/15Frames/test_set.dat","test")
# test_set_data=test_set_data[:datasize("test",batchsz)]
print(1)

1


In [7]:
validation_set_data=getdata("/jf-training-home/datasets_rw/issrlab-dataset2/15Frames/FaceandAudio/FaceAndAudiovalid_set.dat","valid")
validation_set_data=validation_set_data[:datasize("valid",batchsz)]
print(1)

1


In [8]:
train_set_data=getdata("/jf-training-home/datasets_rw/issrlab-dataset2/15Frames/FaceandAudio/FaceAndAudiotrain_set.dat","train")
train_set_data=train_set_data[:datasize("train",batchsz)]
print(1)

1


In [9]:
def imagecrop(imagedata,batchsz):
    # convert_tensor= transforms.ToTensor()
    output=[]
    for j in range(batchsz):#비디오 데이터 batchsz(9) 개 있다
        firstoutput=[]
        for i in range(len(imagedata[j])):#imagedata에 15개의 프레임이 있다
            # temp = convert_tensor(imagedata[i])
            #이미지를 나누
            temp = imagedata[j][i].unfold(1,112,112).unfold(2,112,112)
            temp = rearrange(temp, 'c h1 w1 h w -> (h1 w1) c h w')
            firstoutput.append(temp)
        firstoutput = torch.stack(firstoutput,0)
        output.append(firstoutput)
    output = torch.stack(output,0)
    return output
print(1)

1


In [10]:
def reshape_to_expected_input(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    x0_list = []
    x1_list = []
    x2_list = []
    for i in range(0, len(dataset)):
        x0_list.append(dataset[i][0])
        x1_list.append(dataset[i][1])
        x2_list.append(dataset[i][2])
        print(len(x0_list))
        print(len(x1_list))
        print(len(x2_list))
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list))
print(1)

1


In [11]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [12]:
class ChalearnDataset(Dataset):
    def __init__(self,audiodata,imagedata,tagdata,transform=None):
        self.audiodata=audiodata
        self.imagedata=imagedata
        self.tagdata=tagdata
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.imagedata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        audiodata = self.audiodata[idx]
        audiodata = torch.FloatTensor(audiodata)
        image_data=self.imagedata[idx]
        imagedata1=torch.FloatTensor(image_data)
        # image_data=image_data.reshape(15,3,224,224)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return audiodata,imagedata1,big_five_sorces
print(1)

1


In [13]:
num_workerssz = 20
lr = 3e-05
epochs = 120
#device = torch.device('cuda')
# train_set_data = reshape_to_expected_input(train_set_data)
# val_set_data = reshape_to_expected_input(validation_set_data)
print(1)

1


In [14]:
train_set_data = ChalearnDataset(audiodata = train_set_data[0], imagedata=train_set_data[1],tagdata=train_set_data[2],transform=transform)
val_set_data = ChalearnDataset(audiodata = validation_set_data[0], imagedata=validation_set_data[1],tagdata=validation_set_data[2],transform=transform)
train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

1


In [15]:
model=SwinTransformer3D()#num_frames=frame_nums,batchsz = batchsz
model.to(device)
criterion = torch.nn.L1Loss().to(device)  # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flage==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    # criterion.load_state_dict(checkpoint["loss"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time stride=10
number of patches=131


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3559: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


1


In [16]:
# from torchinfo import summary

# summary(model, input_size = (8,15,224,224,3), col_names = ['input_size','output_size','num_params'])

In [17]:
with torch.cuda.device(0):
    for i in range(start_epoch, epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        
        for audiodata, image_data, big_five_data in tqdm(train_dataloader):
            # image_data=image_data.permute(0,4,1,2,3)
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            # image_data=imagecrop(image_data,batchsz)
            # torch.Size([batchsz, 15, 4 , 3 , 112 , 112])
            # torch.Size([batchsz, 3, 15, 224, 224])
            audiodata = audiodata.to(device)
            image_data = image_data.to(device)
            #torch.Size([64, 5])
            big_five_data = big_five_data.to(device)
            optimizer.zero_grad()  # 기울기가 0이 됩니다.
            hypothesis = model(image_data,audiodata)  # 모델의 예측 결과를 저장합니다.
            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            loss.backward()  # 역방향 전파입니다.
            optimizer.step()  # 매개 변수를 업데이트합니다
            train_avg_loss += loss  # 훈련 손실의 평균치입니다
        train_avg_loss=train_avg_loss/len(train_dataloader)
        
        with torch.no_grad():#validate
            for audiodata, image_data, big_five_data in tqdm(val_dataloader):
                # image_data=image_data.permute(0,4,1,2,3)
                big_five_data=big_five_data.permute(0,2,1)
                big_five_data=big_five_data.squeeze()
                audiodata = audiodata.to(device)
                # image_data=imagecrop(image_data,batchsz)
                image_data = image_data.to(device)
                big_five_data = big_five_data.to(device)
                hypothesis = model(image_data, audiodata)
                val_loss = criterion(hypothesis, big_five_data)
                val_avg_loss += val_loss
            val_avg_loss=val_avg_loss/len(val_dataloader)
        # 每个epoch打印训练和验证的平均损失
        # print("Epoch {}, Train 1-MAE: {:.4f}, Val 1-MAE: {:.4f}".format(i + 1, 1-train_avg_loss, 1-val_avg_loss))

        torch.cuda.empty_cache()
        # if (1-val_avg_loss)>=(1-max_value):
        max_value=val_avg_loss
        start_epoch+=1
        if (i + 1) % 5 == 0:
            torch.save({
                    'epoch': i+1,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'loss': val_avg_loss,
                }, save_model_file_path.format('model',start_epoch,'pth'))
        logger.debug('Epoch: {} , 1-MAE: {:.4f} , 1-train_avg_loss:{:.4f} , 1-val_avg_loss:{:.4f} , MAX Value:{:.4f} , MAX Value Epoch:{}'.format(i+1, 1-val_avg_loss, 1-train_avg_loss, 1-val_avg_loss, max_value, start_epoch))
    

100%|██████████| 125/125 [00:34<00:00,  3.62it/s]
[2024-01-16 10:50:41,907]::<ipython-input-17-d6a636970e37>::DEBUG::Epoch: 1 , 1-MAE: 0.8736 , 1-train_avg_loss:0.8557 , 1-val_avg_loss:0.8736 , MAX Value:0.1264 , MAX Value Epoch:1
100%|██████████| 125/125 [00:34<00:00,  3.63it/s]
[2024-01-16 10:55:49,164]::<ipython-input-17-d6a636970e37>::DEBUG::Epoch: 2 , 1-MAE: 0.8936 , 1-train_avg_loss:0.8797 , 1-val_avg_loss:0.8936 , MAX Value:0.1064 , MAX Value Epoch:2
100%|██████████| 125/125 [00:34<00:00,  3.62it/s]
[2024-01-16 11:00:56,078]::<ipython-input-17-d6a636970e37>::DEBUG::Epoch: 3 , 1-MAE: 0.8833 , 1-train_avg_loss:0.8866 , 1-val_avg_loss:0.8833 , MAX Value:0.1167 , MAX Value Epoch:3
100%|██████████| 125/125 [00:34<00:00,  3.60it/s]
[2024-01-16 11:06:03,750]::<ipython-input-17-d6a636970e37>::DEBUG::Epoch: 4 , 1-MAE: 0.8970 , 1-train_avg_loss:0.8956 , 1-val_avg_loss:0.8970 , MAX Value:0.1030 , MAX Value Epoch:4
100%|██████████| 125/125 [00:34<00:00,  3.62it/s]
[2024-01-16 11:11:19,457]:

In [18]:
# from torchinfo import summary

# summary(model, input_size = (4,15,4,3,224,224))